In [1]:
!pip install pandas
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3227941 sha256=2c307fac1db41cdff901c3167503a6ab1e2a433af9ba9e22a785ad43b1a1cc81
  Stored in directory: /home/codespace/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import numpy as np  
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from surprise import KNNWithMeans
from surprise import SVD, SVDpp
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

import time

In [3]:
start_time = time.time()
df = pd.read_csv("OrderHistory.csv")
df2 = pd.read_csv("ProductsList.csv")
df.head()
computational_time = time.time() - start_time
print('Done in %0.3fs' %(computational_time))

Done in 0.985s


In [4]:
df1 = df.dropna()
df3 = df2.dropna()

In [6]:
df1['Score'].value_counts()


Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [7]:
def map_score(score):
    if isinstance(score, str) and score.lower() == 'score':
        return score
    return int(float(score))
df1['Mapped_Score'] = df1['Score'].map(map_score)


/tmp/ipykernel_5419/1642850552.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Mapped_Score'] = df1['Score'].map(map_score)


In [8]:
df1['Mapped_Score'].value_counts()


Mapped_Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [9]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,105615
1,4,24044
2,1,15373
3,3,12808
4,2,8851


In [10]:
df1 = df1.drop(df1[df1['Score'] == 'Score'].index)
df1['Mapped_Score'].value_counts()


Mapped_Score
5    105615
4     24044
1     15373
3     12808
2      8851
Name: count, dtype: int64

In [11]:
rating_counts = pd.DataFrame(df1['Mapped_Score'].value_counts()).reset_index()
rating_counts.columns = ['Labels', 'Ratings']
rating_counts

,Labels,Ratings
0,5,105615
1,4,24044
2,1,15373
3,3,12808
4,2,8851


In [12]:
users_counts = df1['UserId'].value_counts().rename('users_counts')
users_data   = df1.merge(users_counts.to_frame(),
                                left_on='UserId',
                                right_index=True)

In [13]:
subset_df = users_data[users_data.users_counts>=3]
subset_df

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score,users_counts
9,10,7,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Alisha Solid Women's Cycling Shorts,Alisha,5,3
28,29,11,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Ladela Bellies,Ladela,5,3
195370,195371,24936,A3HDKO7OW0QNK4,5,60,1,Madhya Pradesh,Twella Creations Evergreen Glass Necklace,Twella Creations,5,3
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4,7
33369,33370,4035,A18ECVX2RJ7HUE,3,36,1,Arunachal Pradesh,"UpTown Metal, Alloy Necklace",UpTown,3,7
...,...,...,...,...,...,...,...,...,...,...,...
217259,217260,27680,A9MJ0YE1HFQF5,5,30,0,Andhra Pradesh,Affinity Jewellers Just like diamonds Cubic Zi...,Affinity Jewellers,5,3
218888,218889,27898,A9MJ0YE1HFQF5,5,30,0,Andhra Pradesh,Ace HEADPHONE151ONTO Stereo Dynamic Headphone ...,Ace,5,3
220496,220497,28094,A1KDIFVBAT4MWE,3,55,0,Tripura,Vetra E00TY112 Multifunction Car Tray For Maru...,Vetra,3,3
220497,220498,28094,A1KDIFVBAT4MWE,3,55,0,Tripura,Vetra E00TY112 Multifunction Car Tray For Maru...,Vetra,3,3


In [14]:
product_rating_counts = subset_df['ProductId'].value_counts().rename('product_rating_counts')
product_rating_data   = subset_df.merge(product_rating_counts.to_frame(),
                                left_on='ProductId',
                                right_index=True)
product_rating_data = product_rating_data[product_rating_data.product_rating_counts >= 5]
product_rating_data

,Unnamed: 0,ProductId,UserId,Score,Age,Gender,State,Product_name,Brand,Mapped_Score,users_counts,product_rating_counts
12,13,9,A18ECVX2RJ7HUE,4,36,1,Arunachal Pradesh,"dilli bazaaar Bellies, Corporate Casuals, Casuals",dilli bazaaar,4,7,5
13,14,9,A2MUGFV2TDQ47K,5,60,0,Manipur,Alisha Solid Women's Cycling Shorts,Alisha,5,49,5
17,18,9,A2A9X58G2GTBLP,5,68,1,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,16,5
22,23,9,AJ613OLZZUG7V,5,47,0,Rajasthan,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
24,25,9,A3FONPR03H3PJS,5,56,0,Chhattisgarh,Alisha Solid Women's Cycling Shorts,Alisha,5,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
173337,173338,22245,A16H248EAOBV6U,5,15,1,Jharkhand,PrettySecrets Fashion Women's Push-up Bra,Brown,5,4,5
173338,173339,22245,A3BGO7U46LWLUR,5,29,1,Punjab,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173339,173340,22245,A2SCOK1P7Q9N9L,5,75,1,Andhra Pradesh,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5
173340,173341,22245,A116SXAVY89LJI,5,14,0,Gujarat,Madaam NonPadded Women's Tube Bra,Multicolor,5,4,5


In [15]:
amazon_df = product_rating_data.copy()

In [16]:
panda_data = amazon_df.drop(['users_counts', 'product_rating_counts'], axis=1)

In [17]:
k = 10
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(panda_data[['UserId', 'ProductId', 'Mapped_Score']], reader)

In [18]:
trainset, testset = train_test_split(surprise_data, test_size=.30, random_state=42)

In [22]:
panda_data_grouped = panda_data.groupby('ProductId').agg({'UserId': 'count'}).reset_index()
panda_data_grouped.rename(columns = {'UserId': 'Mapped_Score'},inplace=True)
panda_data_grouped.head()

,ProductId,Mapped_Score
0,9,5
1,13,7
2,34,5
3,313,7
4,318,7


In [24]:
panda_data_sort = panda_data_grouped.sort_values(['Mapped_Score', 'ProductId'], ascending = [0,1]) 

In [26]:
#Generate a recommendation rank based upon score 
panda_data_sort['Rank'] = panda_data_sort['Mapped_Score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = panda_data_sort.head(k) 
popularity_recommendations 

,ProductId,Mapped_Score,Rank
878,17169,440,1.0
954,18099,440,2.0
445,7942,422,3.0
340,5674,395,4.0
613,11272,364,5.0
895,17398,258,6.0
1131,21517,258,7.0
30,1076,254,8.0
109,2273,253,9.0
165,2986,250,10.0


In [27]:
import warnings
warnings.filterwarnings('ignore')
def recommend(userId):     
    user_recommendations = popularity_recommendations 
    user_recommendations['userID'] = userId 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
    return user_recommendations 


In [30]:
find_recom = [15]   # This list is user choice.
recommendation_data = {}
for i in find_recom:
    print("Here is the recommendation for the userId: %d\n" %(i))
    print(recommend(i))
    print("\n") 

Here is the recommendation for the userId: 15

      userID  ProductId  Mapped_Score  Rank
878       15      17169           440   1.0
954       15      18099           440   2.0
445       15       7942           422   3.0
340       15       5674           395   4.0
613       15      11272           364   5.0
895       15      17398           258   6.0
1131      15      21517           258   7.0
30        15       1076           254   8.0
109       15       2273           253   9.0
165       15       2986           250  10.0




In [35]:
find_recom = [15]   # This list is user choice.
recommendation_data = {}

for i in find_recom:
    recommendations_df = recommend(i)
    recommendations_list = recommendations_df.to_dict(orient='records')
    recommendation_data[str(i)] = recommendations_list

    print("Here is the recommendation for the userId: %d\n" % (i))
    print(recommendations_df)
    print("\n")

# Save recommendations to a JSON file
with open('recommendations.json', 'w') as json_file:
    json.dump(recommendation_data, json_file, indent=4, default=str)  # Convert non-serializable objects to string

print("Recommendations saved to 'recommendations.json' file.")






Here is the recommendation for the userId: 15

      userID  ProductId  Mapped_Score  Rank
878       15      17169           440   1.0
954       15      18099           440   2.0
445       15       7942           422   3.0
340       15       5674           395   4.0
613       15      11272           364   5.0
895       15      17398           258   6.0
1131      15      21517           258   7.0
30        15       1076           254   8.0
109       15       2273           253   9.0
165       15       2986           250  10.0


Recommendations saved to 'recommendations.json' file.
